In [1]:
# Import libraries and initialize constants 
import numpy as np
import scipy as sp 
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
## Constants 
hbar = 1.05*(10**(-34))
pi = 3.14159 
c = 3*(10**(8))
kb = 8.617*(10**(-5))
kb_SI = 1.38*(10**(-23))
qe = 1.6*(10**(-19)) 
eps0 = 8.85*(10**(-12))  
m0 = 9*(10**(-31)) 
a_lat = 3.1*(10**(-10)) 
eV2meter = 1.97*(10**(-7)) ##in meter
eV2sec = 6.58*(10**(-16))
eV2mass = 1.78*(10**(-36))

In [ ]:
# Core physiscs funcitons (Assuming parabolic 2DEGs)
def get_fermi_circle(k_rad, nk, nthet):##ok 
    ##Input: radius of 2D Fermi circle (float), size of k array (int), size of theta array (int)
    ##Output: grid of k points in Fermi circle (array)
    kx_list = []
    ky_list = []
    k_range = np.linspace(0 ,k_rad, int(nk))
    thet_range = np.linspace(0, 2*pi, int(nthet))  
    for i in range(0,nk):
        for j in range(0,nthet): 
            kx_list.append(k_range[i]*np.cos(thet_range[j]))
            ky_list.append(k_range[i]*np.sin(thet_range[j]))
    return [np.asarray(kx_list), np.asarray(ky_list)] 

def get_E_field( ): 
    
    
def get_conduction_band(Ec, mstar, k_vec): ##ok 
    ##Input: Conduction band offset (float), effective mass (float), k_point to evaluate at (array)
    ##Output: kmesh (array), conduction band (array)
    Ek = lambda Ec, mstar, kx, ky: Ec + ((hbar**2)/(2*mstar*m0))*(kx**(2) + ky**(2)) 
    return Ek(Ec, mstar, k_vec[0], k_vec[1])   

def get_group_velocity(mstar, kvec): ##ok 
    ##Input: Conduction band offset (float), effective mass (float),  k_point to evaluate at (array)
    ##Output: kmesh (array), conduction band (array)
    vg = lambda mstar, kvec: [((hbar**(2))/(mstar*m0))*kvec[0], ((hbar**(2))/(mstar*m0))*kvec[1]]
    return np.asarray(vg(mstar, k_vec)) 

def get_fermi0(Ek, Ef, T): ##ok 
    ##Input: Ek to evaluate at(float), Fermi level(float), temperature (float) 
    ##Output: equillibrium Fermi funciton 
    f0 = lambda Ek, Ef, T: (1 + np.exp((Ek-Ef)/(kb*T)))**(-1) 
    return f0(Ek, Ef, T)

def get_fermi_RTA(Ek, vg, Ef, T, E_field, tau):
    ##Input: Ek to evaluate at(float), Fermi level(float), temperature (float), applied field (list), scattering time
    ##Output: perturbed Fermi funcion from RTA (float)
    df0 = lambda Ek, Ef, T: (np.exp((Ek-Ef)/(kb*T))/kb*T)*(1 + np.exp((Ek-Ef)/(kb*T)))**(-2)
    f = lambda Ek, vg, Ef, T, E_field, tau: get_fermi0(Ek, Ef, T) + (qe*tau/hbar)*np.dot(vg, E_field)*df0(Ek, Ef, T)  
    return f(Ek, vg, Ef, T, E_field, tau) 

def get_bose_distribution(energy, T):## ok 
    return (np.exp(energy/(kb*T))-1)**(-1)
    
def compute_rate_acoustic(Dac, mstar, k0, kf, rho, vs, energ_ph,T):## ok 
    q = np.linalg.norm(k0-kf)
    mat_elem_sq = lambda Dac, rho, q, vs: ((Dac**(2))*hbar)/(2*rho*vs*q)
    nq = get_bose_distribution(energ_ph, T)
    delE = np.absolute(get_conduction_band(0, mstar, kf) - get_conduction_band(0, mstar, k0))
    
    if(delE*(1/qe) < energ_ph):
        return 0  
    mat_elem = mat_elem_sq(Dac, rho, q, vs) 
    return (2*pi*mat_elem*nq)

def compute_rate_polar_optical(Dac, mstar, k0, kf, rho, vs, energ_ph,T):## ok 
    q = np.linalg.norm(k0-kf)
    mat_elem_sq = lambda Dac, rho, q, vs: ((Dac**(2))*hbar)/(2*rho*vs*q)
    nq = get_bose_distribution(energ_ph, T)
    delE = np.absolute(get_conduction_band(0, mstar, kf) - get_conduction_band(0, mstar, k0))
    
    if(delE*(1/qe) < energ_ph):
        return 0  
    mat_elem = mat_elem_sq(Dac, rho, q, vs) 
    return (2*pi*mat_elem*nq)

def compute_rate_ionized_impurities(Dac, mstar, k0, kf, rho, vs, energ_ph,T):## ok 
    q = np.linalg.norm(k0-kf)
    mat_elem_sq = lambda Dac, rho, q, vs: ((Dac**(2))*hbar)/(2*rho*vs*q)
    nq = get_bose_distribution(energ_ph, T)
    delE = np.absolute(get_conduction_band(0, mstar, kf) - get_conduction_band(0, mstar, k0))
    
    if(delE*(1/qe) < energ_ph):
        return 0  
    mat_elem = mat_elem_sq(Dac, rho, q, vs) 
    return (2*pi*mat_elem*nq)

def compute_rate_dislocations(Dac, mstar, k0, kf, rho, vs, energ_ph,T):  
    q = np.linalg.norm(k0-kf)
    mat_elem_sq = lambda Dac, rho, q, vs: ((Dac**(2))*hbar)/(2*rho*vs*q)
    nq = get_bose_distribution(energ_ph, T)
    delE = np.absolute(get_conduction_band(0, mstar, kf) - get_conduction_band(0, mstar, k0))
    
    if(delE*(1/qe) < energ_ph):
        return 0  
    mat_elem = mat_elem_sq(Dac, rho, q, vs) 
    return (2*pi*mat_elem*nq)

# Monte carlo specific functions 
def evolvek_h(del_t, E_field, k0):  
    del_k = (qe*del_t/hbar)*E_field 
    return k0 + del_k

def scatter_prob_h(del_t, E_field, k0): 
    
def pick_k_h(k0, kmesh, energ_ph, Dac, mstar, rho, vs, T):   

def pick_mechanism_h(k0, kmesh, energ_ph, Dac, mstar, rho, vs, T): 
    

def monte_carlo_main(kmesh, Ek, energ_ph, Dac, mstar, rho, vs, T, E_field, del_t, Tmax):

def compute_autocorrelation(): 

def compute_spectral_density(): 

